In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import os
dir = '/Users/signapoop/Desktop/autotune'
os.chdir(dir)
from autotune.core.HyperbandOptimiser import HyperbandOptimiser
from autotune.core.RandomOptimiser import RandomOptimiser
from autotune.core.TpeOptimiser import TpeOptimiser
from autotune.util.best_value import best_value

In [3]:
path = '/Users/signapoop/Desktop/'
os.chdir(path)

In [23]:
checkpoints = []
Y = []
Y_best = []
with open('log_4.txt') as f:  
    for line in f:
        if "num iteration" in line:
            chunks = line.split(" ")
            checkpoints.append(float(chunks[5][:-2]))
            Y.append(float(chunks[7][:-1]))
            Y_best.append(float(chunks[9]))
            # print line.split(" ")

In [24]:
tpe_opt = TpeOptimiser()
tpe_opt.checkpoints = checkpoints
tpe_opt.Y = Y  
tpe_opt.Y_best = Y_best 

In [25]:
filename = '/Users/signapoop/Desktop/' + 'tpe_10_results.pkl'
with open(filename, 'wb') as f:
    pickle.dump([tpe_opt], f)

In [19]:
tpe_opt.checkpoints

[291.93,
 553.9,
 1015.38,
 1202.71,
 1396.84,
 1655.55,
 2122.18,
 2419.39,
 2638.86,
 3046.56,
 3357.83,
 3697.75,
 3906.27,
 4210.74,
 4653.19,
 4935.04,
 5139.63,
 5422.45,
 5710.94,
 6030.55,
 6433.1,
 6787.31,
 7397.08,
 7715.14,
 8002.42,
 8328.56,
 8595.15,
 8799.98,
 9094.85,
 9300.24]

In [74]:
path = '/Users/signapoop/Desktop/results/' + 'cifar_9hps_rand_40'
os.chdir(path)
file = open("results.pkl",'rb')
object_file = pickle.load(file)
file.close()

In [75]:
random_big = object_file[0]
len(random_big.checkpoints)

40

In [76]:
random_big.checkpoints = [0] + random_big.checkpoints
new_checkpoints = []
for i, checkpoint in enumerate(random_big.checkpoints):
    if i%8 == 1:
        seq = random_big.checkpoints[i:i+8]
        for j in range(len(seq)):
            seq[j] = seq[j] - random_big.checkpoints[i-1]
        new_checkpoints.append(seq)

In [77]:
new_checkpoints

[[380.69688987731934,
  575.3999969959259,
  853.56290102005,
  1057.943685054779,
  1237.1627209186554,
  1464.783919095993,
  1739.3638820648193,
  1936.3104510307312],
 [285.899493932724,
  726.7349028587341,
  924.3932549953461,
  1110.673220872879,
  1585.5193140506744,
  1845.3033349514008,
  2257.279387950897,
  2465.54008102417],
 [182.73752903938293,
  651.6970808506012,
  869.7835869789124,
  1099.1519210338593,
  1389.4739909172058,
  1728.450530052185,
  2016.8266849517822,
  2304.483670949936],
 [495.76940298080444,
  902.767462015152,
  1105.5819339752197,
  1436.0530769824982,
  1742.4861969947815,
  2051.939785003662,
  2438.667890071869,
  2640.2089738845825],
 [337.02306604385376,
  525.9153151512146,
  709.5576450824738,
  937.4078421592712,
  1150.15278506279,
  1333.3333129882812,
  1592.7822411060333,
  2005.0112540721893]]

In [85]:
n = 5
random = []
for i in range(0,n):

    problem_text = "cifar_9hps_" + str(i+1)
    path = '/Users/signapoop/Desktop/results/' + problem_text
    os.chdir(path)
    file = open("results.pkl",'rb')
    object_file = pickle.load(file)
    file.close()
    hb_opt = object_file[0]
    random_opt = object_file[1]
    
    # Append y values
    random_opt.Y = random_opt.Y + random_big.Y[i*8:(i+1)*8]
    
    # Append checkpoints
    seq = new_checkpoints[i]
    seq = [s + random_opt.checkpoints[-1] for s in seq]
    random_opt.checkpoints = random_opt.checkpoints + seq
    
    # Recompute y_best
    random_opt.Y_best = best_value(random_opt.Y)
    random.append(random_opt)
    
    filename = 'results_new.pkl'
    with open(filename, 'wb') as f:
        pickle.dump([hb_opt, random_opt], f)
    

In [84]:
for r in random:
    print(len(r.checkpoints))
    print(r.Y)
    #print(r.checkpoints)
    #print(r.Y)

25
[0.5825, 0.24329999999999996, 0.9, 0.2168, 0.3909, 0.5926, 0.8114, 0.43720000000000003, 0.2952, 0.6668000000000001, 0.6799, 0.5381, 0.7765, 0.9, 0.4507, 0.2653, 0.46299999999999997, 0.26049999999999995, 0.2672, 0.19469999999999998, 0.4014, 0.3671, 0.8168, 0.5525, 0.6737]
25
[0.44389999999999996, 0.2389, 0.5379, 0.3033, 0.18369999999999997, 0.8987, 0.2238, 0.6466000000000001, 0.2136, 0.32909999999999995, 0.7010000000000001, 0.2106, 0.1694, 0.5856, 0.43589999999999995, 0.35309999999999997, 0.2996, 0.7613, 0.20399999999999996, 0.2117, 0.518, 0.17779999999999996, 0.24270000000000003, 0.9, 0.38959999999999995]
21
[0.39339999999999997, 0.7705, 0.5925, 0.8607, 0.6179, 0.4104, 0.22629999999999995, 0.25149999999999995, 0.9, 0.4305, 0.563, 0.5886, 0.23929999999999996, 0.27480000000000004, 0.6289, 0.8636, 0.5538000000000001, 0.30979999999999996, 0.20589999999999997, 0.19589999999999996, 0.19889999999999997]
26
[0.19240000000000002, 0.635, 0.3679, 0.47850000000000004, 0.45620000000000005, 0.265